In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import time
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

In [3]:
df_train = pd.read_csv("../input/Cascade_cup/train_age_dataset.csv")
df_test = pd.read_csv("../input/Cascade_cup/test_age_dataset.csv")

In [4]:
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

In [5]:
cat_columns = ['tier','gender']

In [6]:
encoded_train = ohe.fit_transform(df_train[cat_columns])
encoded_test = ohe.transform(df_test[cat_columns])
encoded_df_train = pd.DataFrame(encoded_train,columns=ohe.get_feature_names(cat_columns))
encoded_df_train.index = df_train.index
encoded_df_test = pd.DataFrame(encoded_test,columns=ohe.get_feature_names(cat_columns))
encoded_df_test.index = df_test.index
train = pd.concat([df_train.drop(cat_columns,axis=1),encoded_df_train],axis=1)
test = pd.concat([df_test.drop(cat_columns,axis=1),encoded_df_test],axis=1)

In [ ]:
train['Unnamed: 0'].plot(kind='kde')

In [7]:
feature_cols = [col for col in train.columns.tolist() if col not in ['age_group']]
target_cols = ['age_group']

In [ ]:
skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)

for i, (trn, val) in enumerate(skf.split(train[target_cols], train[target_cols])):
    train.loc[val, 'kfold'] = i
train['kfold'] = train['kfold'].astype(int)

In [8]:
from sklearn.feature_selection import RFECV

In [ ]:
from sklearn.utils import class_weight

In [9]:
estimator = RandomForestClassifier(n_estimators=100, n_jobs=-1, class_weight='balanced', random_state=42)

In [ ]:
estimator = xgb.XGBClassifier(n_estimators=300, random_state=42, eta=0.1, n_jobs=-1)

In [11]:
selector = RFECV(estimator, scoring='f1_weighted',step=1, cv=5, verbose=1, n_jobs=-1, min_features_to_select=10)

In [12]:
selector.fit(train[feature_cols], train[target_cols])

Fitting estimator with 29 features.


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_rfe.py:196: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)


Fitting estimator with 28 features.


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_rfe.py:196: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)


Fitting estimator with 27 features.


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_rfe.py:196: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)


Fitting estimator with 26 features.


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_rfe.py:196: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_rfe.py:231: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X[:, features], y)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_rfe.py:573: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(self.transform(X), y)


RFECV(cv=5,
      estimator=RandomForestClassifier(class_weight='balanced', n_jobs=-1,
                                       random_state=42),
      min_features_to_select=10, n_jobs=-1, scoring='f1_weighted', verbose=1)

In [13]:
selector.score(train[feature_cols], train[target_cols])

1.0

In [14]:
selector.n_features_

25

In [15]:
selector.ranking_

array([1, 1, 1, 1, 1, 1, 5, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 3, 1, 1])

In [18]:
feature_cols

['Unnamed: 0',
 'userId',
 'following_rate',
 'followers_avg_age',
 'following_avg_age',
 'max_repetitive_punc',
 'num_of_hashtags_per_action',
 'emoji_count_per_action',
 'punctuations_per_action',
 'number_of_words_per_action',
 'avgCompletion',
 'avgTimeSpent',
 'avgDuration',
 'avgComments',
 'creations',
 'content_views',
 'num_of_comments',
 'weekends_trails_watched_per_day',
 'weekdays_trails_watched_per_day',
 'slot1_trails_watched_per_day',
 'slot2_trails_watched_per_day',
 'slot3_trails_watched_per_day',
 'slot4_trails_watched_per_day',
 'avgt2',
 'tier_1',
 'tier_2',
 'tier_3',
 'gender_1',
 'gender_2']

In [ ]:
selector.grid_scores_

In [ ]:
#'num_of_hashtags_per_action', 'emoji_count_per_action', tier_1, tier_3

In [19]:
preds = selector.predict(test)
preds

array([1, 1, 1, ..., 4, 1, 1])

In [21]:
preds_df = pd.Series(preds, name='prediction')

In [22]:
preds_df.to_csv('Submission.csv', index=False)

In [ ]:
# num_of_hashtags_per_action, emoji_count_per_action, punctuations_per_action, avgComments,
# tier_1, tier_2, tier_3, gender_1, gender_2